In [ ]:
from matplotlib import pyplot as plt
from tensorflow import keras

import tensorflow as tf
import pandas as pd
import numpy as np
import random
import math
import keras.backend as K

import cv2
import os

WIDTH, HEIGHT = 1024, 1024
NUM_CLASSES = 6 # 0. Background, 1, Exudatas(Hard + Soft), 2. Red Dot + Hemorrhages + Microaneurysms
BATCH_SIZE = 1


EPOCHS = 50

# On Colab
MODEL_DIR = "./Diabetic Retinopathy/"
BASE_DIR = './DR_data/'

palette = [[0],[100],[200]]#100：(green)Red Dot + Hemorrhages + Microaneurysms 200：(yellow)Exudatas(Hard + Soft)
category_types = ["Background", "EX", "RHM"]

# Construct Model

In [ ]:
import torchfile  # pip install torchfile

#From https://github.com/dalgu90/resnet-18-tensorflow/blob/master/extract_torch_t7.py

T7_PATH = './resnet-18.t7'

In [ ]:
def load_pretrained_weights(sess, T7_PATH=T7_PATH):
    # Open ResNet-18 torch checkpoint
    print('Open ResNet-18 torch checkpoint: %s' % T7_PATH)
    o = torchfile.load(T7_PATH)

    # Load weights in a brute-force way
    print('Load weights in a brute-force way')
    conv1_weights = o.modules[0].weight
    conv1_bn_gamma = o.modules[1].weight
    conv1_bn_beta = o.modules[1].bias
    conv1_bn_mean = o.modules[1].running_mean
    conv1_bn_var = o.modules[1].running_var

    conv2_1_weights_1  = o.modules[4].modules[0].modules[0].modules[0].modules[0].weight
    conv2_1_bn_1_gamma = o.modules[4].modules[0].modules[0].modules[0].modules[1].weight
    conv2_1_bn_1_beta  = o.modules[4].modules[0].modules[0].modules[0].modules[1].bias
    conv2_1_bn_1_mean  = o.modules[4].modules[0].modules[0].modules[0].modules[1].running_mean
    conv2_1_bn_1_var   = o.modules[4].modules[0].modules[0].modules[0].modules[1].running_var
    conv2_1_weights_2  = o.modules[4].modules[0].modules[0].modules[0].modules[3].weight
    conv2_1_bn_2_gamma = o.modules[4].modules[0].modules[0].modules[0].modules[4].weight
    conv2_1_bn_2_beta  = o.modules[4].modules[0].modules[0].modules[0].modules[4].bias
    conv2_1_bn_2_mean  = o.modules[4].modules[0].modules[0].modules[0].modules[4].running_mean
    conv2_1_bn_2_var   = o.modules[4].modules[0].modules[0].modules[0].modules[4].running_var
    conv2_2_weights_1  = o.modules[4].modules[1].modules[0].modules[0].modules[0].weight
    conv2_2_bn_1_gamma = o.modules[4].modules[1].modules[0].modules[0].modules[1].weight
    conv2_2_bn_1_beta  = o.modules[4].modules[1].modules[0].modules[0].modules[1].bias
    conv2_2_bn_1_mean  = o.modules[4].modules[1].modules[0].modules[0].modules[1].running_mean
    conv2_2_bn_1_var   = o.modules[4].modules[1].modules[0].modules[0].modules[1].running_var
    conv2_2_weights_2  = o.modules[4].modules[1].modules[0].modules[0].modules[3].weight
    conv2_2_bn_2_gamma = o.modules[4].modules[1].modules[0].modules[0].modules[4].weight
    conv2_2_bn_2_beta  = o.modules[4].modules[1].modules[0].modules[0].modules[4].bias
    conv2_2_bn_2_mean  = o.modules[4].modules[1].modules[0].modules[0].modules[4].running_mean
    conv2_2_bn_2_var   = o.modules[4].modules[1].modules[0].modules[0].modules[4].running_var

    conv3_1_weights_skip = o.modules[5].modules[0].modules[0].modules[1].weight
    conv3_1_weights_1  = o.modules[5].modules[0].modules[0].modules[0].modules[0].weight
    conv3_1_bn_1_gamma = o.modules[5].modules[0].modules[0].modules[0].modules[1].weight
    conv3_1_bn_1_beta  = o.modules[5].modules[0].modules[0].modules[0].modules[1].bias
    conv3_1_bn_1_mean  = o.modules[5].modules[0].modules[0].modules[0].modules[1].running_mean
    conv3_1_bn_1_var   = o.modules[5].modules[0].modules[0].modules[0].modules[1].running_var
    conv3_1_weights_2  = o.modules[5].modules[0].modules[0].modules[0].modules[3].weight
    conv3_1_bn_2_gamma = o.modules[5].modules[0].modules[0].modules[0].modules[4].weight
    conv3_1_bn_2_beta  = o.modules[5].modules[0].modules[0].modules[0].modules[4].bias
    conv3_1_bn_2_mean  = o.modules[5].modules[0].modules[0].modules[0].modules[4].running_mean
    conv3_1_bn_2_var   = o.modules[5].modules[0].modules[0].modules[0].modules[4].running_var
    conv3_2_weights_1  = o.modules[5].modules[1].modules[0].modules[0].modules[0].weight
    conv3_2_bn_1_gamma = o.modules[5].modules[1].modules[0].modules[0].modules[1].weight
    conv3_2_bn_1_beta  = o.modules[5].modules[1].modules[0].modules[0].modules[1].bias
    conv3_2_bn_1_mean  = o.modules[5].modules[1].modules[0].modules[0].modules[1].running_mean
    conv3_2_bn_1_var   = o.modules[5].modules[1].modules[0].modules[0].modules[1].running_var
    conv3_2_weights_2  = o.modules[5].modules[1].modules[0].modules[0].modules[3].weight
    conv3_2_bn_2_gamma = o.modules[5].modules[1].modules[0].modules[0].modules[4].weight
    conv3_2_bn_2_beta  = o.modules[5].modules[1].modules[0].modules[0].modules[4].bias
    conv3_2_bn_2_mean  = o.modules[5].modules[1].modules[0].modules[0].modules[4].running_mean
    conv3_2_bn_2_var   = o.modules[5].modules[1].modules[0].modules[0].modules[4].running_var

    conv4_1_weights_skip = o.modules[6].modules[0].modules[0].modules[1].weight
    conv4_1_weights_1  = o.modules[6].modules[0].modules[0].modules[0].modules[0].weight
    conv4_1_bn_1_gamma = o.modules[6].modules[0].modules[0].modules[0].modules[1].weight
    conv4_1_bn_1_beta  = o.modules[6].modules[0].modules[0].modules[0].modules[1].bias
    conv4_1_bn_1_mean  = o.modules[6].modules[0].modules[0].modules[0].modules[1].running_mean
    conv4_1_bn_1_var   = o.modules[6].modules[0].modules[0].modules[0].modules[1].running_var
    conv4_1_weights_2  = o.modules[6].modules[0].modules[0].modules[0].modules[3].weight
    conv4_1_bn_2_gamma = o.modules[6].modules[0].modules[0].modules[0].modules[4].weight
    conv4_1_bn_2_beta  = o.modules[6].modules[0].modules[0].modules[0].modules[4].bias
    conv4_1_bn_2_mean  = o.modules[6].modules[0].modules[0].modules[0].modules[4].running_mean
    conv4_1_bn_2_var   = o.modules[6].modules[0].modules[0].modules[0].modules[4].running_var
    conv4_2_weights_1  = o.modules[6].modules[1].modules[0].modules[0].modules[0].weight
    conv4_2_bn_1_gamma = o.modules[6].modules[1].modules[0].modules[0].modules[1].weight
    conv4_2_bn_1_beta  = o.modules[6].modules[1].modules[0].modules[0].modules[1].bias
    conv4_2_bn_1_mean  = o.modules[6].modules[1].modules[0].modules[0].modules[1].running_mean
    conv4_2_bn_1_var   = o.modules[6].modules[1].modules[0].modules[0].modules[1].running_var
    conv4_2_weights_2  = o.modules[6].modules[1].modules[0].modules[0].modules[3].weight
    conv4_2_bn_2_gamma = o.modules[6].modules[1].modules[0].modules[0].modules[4].weight
    conv4_2_bn_2_beta  = o.modules[6].modules[1].modules[0].modules[0].modules[4].bias
    conv4_2_bn_2_mean  = o.modules[6].modules[1].modules[0].modules[0].modules[4].running_mean
    conv4_2_bn_2_var   = o.modules[6].modules[1].modules[0].modules[0].modules[4].running_var

    conv5_1_weights_skip = o.modules[7].modules[0].modules[0].modules[1].weight
    conv5_1_weights_1  = o.modules[7].modules[0].modules[0].modules[0].modules[0].weight
    conv5_1_bn_1_gamma = o.modules[7].modules[0].modules[0].modules[0].modules[1].weight
    conv5_1_bn_1_beta  = o.modules[7].modules[0].modules[0].modules[0].modules[1].bias
    conv5_1_bn_1_mean  = o.modules[7].modules[0].modules[0].modules[0].modules[1].running_mean
    conv5_1_bn_1_var   = o.modules[7].modules[0].modules[0].modules[0].modules[1].running_var
    conv5_1_weights_2  = o.modules[7].modules[0].modules[0].modules[0].modules[3].weight
    conv5_1_bn_2_gamma = o.modules[7].modules[0].modules[0].modules[0].modules[4].weight
    conv5_1_bn_2_beta  = o.modules[7].modules[0].modules[0].modules[0].modules[4].bias
    conv5_1_bn_2_mean  = o.modules[7].modules[0].modules[0].modules[0].modules[4].running_mean
    conv5_1_bn_2_var   = o.modules[7].modules[0].modules[0].modules[0].modules[4].running_var
    conv5_2_weights_1  = o.modules[7].modules[1].modules[0].modules[0].modules[0].weight
    conv5_2_bn_1_gamma = o.modules[7].modules[1].modules[0].modules[0].modules[1].weight
    conv5_2_bn_1_beta  = o.modules[7].modules[1].modules[0].modules[0].modules[1].bias
    conv5_2_bn_1_mean  = o.modules[7].modules[1].modules[0].modules[0].modules[1].running_mean
    conv5_2_bn_1_var   = o.modules[7].modules[1].modules[0].modules[0].modules[1].running_var
    conv5_2_weights_2  = o.modules[7].modules[1].modules[0].modules[0].modules[3].weight
    conv5_2_bn_2_gamma = o.modules[7].modules[1].modules[0].modules[0].modules[4].weight
    conv5_2_bn_2_beta  = o.modules[7].modules[1].modules[0].modules[0].modules[4].bias
    conv5_2_bn_2_mean  = o.modules[7].modules[1].modules[0].modules[0].modules[4].running_mean
    conv5_2_bn_2_var   = o.modules[7].modules[1].modules[0].modules[0].modules[4].running_var

    fc_weights = o.modules[10].weight
    fc_biases = o.modules[10].bias

    model_weights_map = {
        'initial_block/conv1/kernel': conv1_weights,
        'initial_block/bn1/moving_mean': conv1_bn_mean,
        'initial_block/bn1/moving_variance': conv1_bn_var,
        'initial_block/bn1/beta': conv1_bn_beta,
        'initial_block/bn1/gamma': conv1_bn_gamma,

        'eb1/bb1/conv_bn_relu/conv/kernel': conv2_1_weights_1,
        'eb1/bb1/conv_bn_relu/bn/moving_mean': conv2_1_bn_1_mean,
        'eb1/bb1/conv_bn_relu/bn/moving_variance': conv2_1_bn_1_var,
        'eb1/bb1/conv_bn_relu/bn/beta': conv2_1_bn_1_beta,
        'eb1/bb1/conv_bn_relu/bn/gamma': conv2_1_bn_1_gamma,
        'eb1/bb1/conv2/kernel': conv2_1_weights_2,
        'eb1/bb1/bn2/moving_mean': conv2_1_bn_2_mean,
        'eb1/bb1/bn2/moving_variance': conv2_1_bn_2_var,
        'eb1/bb1/bn2/beta': conv2_1_bn_2_beta,
        'eb1/bb1/bn2/gamma': conv2_1_bn_2_gamma,
        'eb1/bb2/conv_bn_relu/conv/kernel': conv2_2_weights_1,
        'eb1/bb2/conv_bn_relu/bn/moving_mean': conv2_2_bn_1_mean,
        'eb1/bb2/conv_bn_relu/bn/moving_variance': conv2_2_bn_1_var,
        'eb1/bb2/conv_bn_relu/bn/beta': conv2_2_bn_1_beta,
        'eb1/bb2/conv_bn_relu/bn/gamma': conv2_2_bn_1_gamma,
        'eb1/bb2/conv2/kernel': conv2_2_weights_2,
        'eb1/bb2/bn2/moving_mean': conv2_2_bn_2_mean,
        'eb1/bb2/bn2/moving_variance': conv2_2_bn_2_var,
        'eb1/bb2/bn2/beta': conv2_2_bn_2_beta,
        'eb1/bb2/bn2/gamma': conv2_2_bn_2_gamma,

        'eb2/bb1/dimension_reduction/kernel': conv3_1_weights_skip,
        'eb2/bb1/conv_bn_relu/conv/kernel': conv3_1_weights_1,
        'eb2/bb1/conv_bn_relu/bn/moving_mean': conv3_1_bn_1_mean,
        'eb2/bb1/conv_bn_relu/bn/moving_variance': conv3_1_bn_1_var,
        'eb2/bb1/conv_bn_relu/bn/beta': conv3_1_bn_1_beta,
        'eb2/bb1/conv_bn_relu/bn/gamma': conv3_1_bn_1_gamma,
        'eb2/bb1/conv2/kernel': conv3_1_weights_2,
        'eb2/bb1/bn2/moving_mean': conv3_1_bn_2_mean,
        'eb2/bb1/bn2/moving_variance': conv3_1_bn_2_var,
        'eb2/bb1/bn2/beta': conv3_1_bn_2_beta,
        'eb2/bb1/bn2/gamma': conv3_1_bn_2_gamma,
        'eb2/bb2/conv_bn_relu/conv/kernel': conv3_2_weights_1,
        'eb2/bb2/conv_bn_relu/bn/moving_mean': conv3_2_bn_1_mean,
        'eb2/bb2/conv_bn_relu/bn/moving_variance': conv3_2_bn_1_var,
        'eb2/bb2/conv_bn_relu/bn/beta': conv3_2_bn_1_beta,
        'eb2/bb2/conv_bn_relu/bn/gamma': conv3_2_bn_1_gamma,
        'eb2/bb2/conv2/kernel': conv3_2_weights_2,
        'eb2/bb2/bn2/moving_mean': conv3_2_bn_2_mean,
        'eb2/bb2/bn2/moving_variance': conv3_2_bn_2_var,
        'eb2/bb2/bn2/beta': conv3_2_bn_2_beta,
        'eb2/bb2/bn2/gamma': conv3_2_bn_2_gamma,

        'eb3/bb1/dimension_reduction/kernel': conv4_1_weights_skip,
        'eb3/bb1/conv_bn_relu/conv/kernel': conv4_1_weights_1,
        'eb3/bb1/conv_bn_relu/bn/moving_mean': conv4_1_bn_1_mean,
        'eb3/bb1/conv_bn_relu/bn/moving_variance': conv4_1_bn_1_var,
        'eb3/bb1/conv_bn_relu/bn/beta': conv4_1_bn_1_beta,
        'eb3/bb1/conv_bn_relu/bn/gamma': conv4_1_bn_1_gamma,
        'eb3/bb1/conv2/kernel': conv4_1_weights_2,
        'eb3/bb1/bn2/moving_mean': conv4_1_bn_2_mean,
        'eb3/bb1/bn2/moving_variance': conv4_1_bn_2_var,
        'eb3/bb1/bn2/beta': conv4_1_bn_2_beta,
        'eb3/bb1/bn2/gamma': conv4_1_bn_2_gamma,
        'eb3/bb2/conv_bn_relu/conv/kernel': conv4_2_weights_1,
        'eb3/bb2/conv_bn_relu/bn/moving_mean': conv4_2_bn_1_mean,
        'eb3/bb2/conv_bn_relu/bn/moving_variance': conv4_2_bn_1_var,
        'eb3/bb2/conv_bn_relu/bn/beta': conv4_2_bn_1_beta,
        'eb3/bb2/conv_bn_relu/bn/gamma': conv4_2_bn_1_gamma,
        'eb3/bb2/conv2/kernel': conv4_2_weights_2,
        'eb3/bb2/bn2/moving_mean': conv4_2_bn_2_mean,
        'eb3/bb2/bn2/moving_variance': conv4_2_bn_2_var,
        'eb3/bb2/bn2/beta': conv4_2_bn_2_beta,
        'eb3/bb2/bn2/gamma': conv4_2_bn_2_gamma,

        'eb4/bb1/dimension_reduction/kernel': conv5_1_weights_skip,
        'eb4/bb1/conv_bn_relu/conv/kernel': conv5_1_weights_1,
        'eb4/bb1/conv_bn_relu/bn': conv5_1_bn_1_mean,
        'eb4/bb1/conv_bn_relu/bn/moving_variance': conv5_1_bn_1_var,
        'eb4/bb1/conv_bn_relu/bn/beta': conv5_1_bn_1_beta,
        'eb4/bb1/conv_bn_relu/bn/gamma': conv5_1_bn_1_gamma,
        'eb4/bb1/conv2/kernel': conv5_1_weights_2,
        'eb4/bb1/bn2/moving_mean': conv5_1_bn_2_mean,
        'eb4/bb1/bn2/moving_variance': conv5_1_bn_2_var,
        'eb4/bb1/bn2/beta': conv5_1_bn_2_beta,
        'eb4/bb1/bn2/gamma': conv5_1_bn_2_gamma,
        'eb4/bb2/conv_bn_relu/conv/kernel': conv5_2_weights_1,
        'eb4/bb2/conv_bn_relu/bn/moving_mean': conv5_2_bn_1_mean,
        'eb4/bb2/conv_bn_relu/bn/moving_variance': conv5_2_bn_1_var,
        'eb4/bb2/conv_bn_relu/bn/beta': conv5_2_bn_1_beta,
        'eb4/bb2/conv_bn_relu/bn/gamma': conv5_2_bn_1_gamma,
        'eb4/bb2/conv2/kernel': conv5_2_weights_2,
        'eb4/bb2/bn2/moving_mean': conv5_2_bn_2_mean,
        'eb4/bb2/bn2/moving_variance': conv5_2_bn_2_var,
        'eb4/bb2/bn2/beta': conv5_2_bn_2_beta,
        'eb4/bb2/bn2/gamma': conv5_2_bn_2_gamma,

    #    'logits/fc/weights': fc_weights,
    #    'logits/fc/biases': fc_biases,
    }

    # Transpose conv and fc weights
    model_weights = {}
    for k, v in model_weights_map.items():
        if len(v.shape) == 4:
            model_weights[k] = np.transpose(v, (2, 3, 1, 0))
        elif len(v.shape) == 2:
            model_weights[k] = np.transpose(v)
        else:
            model_weights[k] = v

    with tf.variable_scope('linknet', reuse=True):
        for k, v in model_weights_map.items():
            sess.run(tf.get_variable(k).assign(v))

In [ ]:
import tensorflow as tf1# import tensorflow.compat.v1 as tf1
#import tensorflow.contrib.slim as slim
import tf_slim as slim
from tensorflow.python.keras.initializers import he_normal
from tf_slim import arg_scope, add_arg_scope

In [ ]:
@add_arg_scope
def conv_bn_relu(x, num_channel, kernel_size, stride,
                 is_training, scope, padding='same', use_bias=False):
    with tf1.compat.v1.variable_scope(scope):
        x = keras.layers.Conv2D(num_channel, [kernel_size, kernel_size],
                             strides=stride, activation=None, #name='conv',
                             padding=padding, use_bias=use_bias,
                             kernel_initializer=he_normal())(x)
        x = keras.layers.BatchNormalization(momentum=0.9,)(x)
                                          #name='bn')
        x = tf1.nn.relu(x, name='relu')
        return x
@add_arg_scope
def basic_block(x, num_channel, kernel_size,
                stride, is_training, scope, padding='same'):
    # Shortcut connection
    in_channel = x.get_shape().as_list()[-1]# 64 64 64
    
    with tf1.compat.v1.variable_scope(scope):
        if in_channel == num_channel:
            if stride == 1:
                shortcut = tf1.identity(x)
            else:
                shortcut = tf1.nn.max_pool(x, [stride, stride], stride,
                                          padding='same')
        else:
            # Considering maxpooling if stride > 1
            shortcut = keras.layers.Conv2D(num_channel, 1, strides=stride,
                                        padding='same', activation=None,
                                        # name='dimension_reduction',
                                        use_bias=False,
                                        kernel_initializer=he_normal())(x)
            # shortcut = tf1.layers.batch_normalization(
            #     shortcut, momentum=0.9, training=is_training,
            #     name='shortcut_bn'
            # )

        x = conv_bn_relu(x, num_channel, kernel_size, stride,
                         is_training=is_training, scope='conv_bn_relu',
                         padding=padding)
        x = keras.layers.Conv2D(num_channel, [kernel_size, kernel_size],
                             strides=1, padding=padding, #name='conv2',
                             use_bias=False, kernel_initializer=he_normal())(x)
        x = keras.layers.BatchNormalization( momentum=0.9,
                                           )(x)#name='bn2')# is_trainging delete
        # Considering add relu to x before addition
        x = x + shortcut
        x = tf1.nn.relu(x)
    return x


@add_arg_scope
def encoder_block(x, num_channel, kernel_size, stride, is_training, scope,
                 padding='same'):
    with tf1.compat.v1.variable_scope(scope):
        x = basic_block(x, num_channel, kernel_size, stride, is_training,
                        scope=scope+'bb1', padding=padding)# (256,256,64) scope='bb1'
        
        x = basic_block(x, num_channel, kernel_size, 1, is_training,
                        scope=scope+'bb2', padding=padding)
       
    return x



@add_arg_scope
def upconv_bn_relu(x, num_channel, kernel_size, stride, is_training,
                   scope, padding='same', use_bias=False):
    with tf1.compat.v1.variable_scope(scope):
        x = keras.layers.Conv2DTranspose( num_channel, [kernel_size, kernel_size],
                                      stride, activation=None,
                                       padding=padding,# name='conv_transpose',
                                      use_bias=False, kernel_initializer=he_normal())(x)
        x = keras.layers.BatchNormalization(
            momentum=0.9, #training=is_training, #name='bn'
        )(x)
        x = tf1.nn.relu(x, name='relu')
    return x


@add_arg_scope
def decoder_block(x, num_channel_m, num_channel_n, kernel_size,
                  stride=1, is_training=True, scope=None, padding='same'):
    with tf1.compat.v1.variable_scope(scope):
        x = upconv_bn_relu(x, num_channel_m // 4, 1, 1,
                           is_training=is_training,
                           scope='conv_transpose_bn_relu1',
                           padding=padding)
        x = upconv_bn_relu(x, num_channel_m // 4, kernel_size, stride,
                           is_training=is_training,
                           scope='conv_transpose_bn_relu2',
                           padding=padding)
        x = upconv_bn_relu(x, num_channel_n, 1, 1,
                           is_training=is_training,
                           scope='conv_transpose_bn_relu3',
                           padding=padding)
#         print("new___final",x.shape)
#         new___final (None, 64, 64, 256)
#         new___final (None, 128, 128, 128)
#         new___final (None, 256, 256, 64)
#         new___final (None, 256, 256, 64)
    return x


@add_arg_scope
def initial_block(x, scope='initial_block',
                  padding='same', use_bias=False):
    with tf1.compat.v1.variable_scope(scope):
        x = keras.layers.Conv2D(64, 7, strides=(2,2), activation=None,
                             name='conv1', padding=padding, use_bias=use_bias,
                             kernel_initializer=he_normal())(x)
#         print("new____new",x.shape)# (512,512,64)
        x = keras.layers.BatchNormalization(
            momentum=0.9, name='bn1'
        )(x)
        x = keras.layers.ReLU(name='relu')(x)
        x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=2,name='maxpool',padding=padding)(x) 
          
    return x

In [ ]:
def linknet(num_classes=NUM_CLASSES, width = WIDTH, height = HEIGHT, reuse=None, is_training=True,
            scope='linknet'):

    inputs = keras.Input(shape=(width, height, 3), name="Image")

    filters = [64, 128, 256, 512]
    filters_m = [64, 128, 256, 512][::-1]
    filters_n = [64, 64, 128, 256][::-1]

    with tf1.compat.v1.variable_scope(scope, reuse=reuse):

            # Encoder
            eb0 = initial_block(inputs, 
                                scope='initial_block')
#             print("eb0.shape",eb0.shape)#eb0.shape (None, 256, 256, 64)
            eb1 = encoder_block(eb0, filters[0], 3, 1, is_training,
                                scope='eb1', padding='same')# (256,256,64)
            ebi = eb1
            ebs = [eb1, ]
            i = 2
            for filter_i in filters[1:]:
                
                ebi = encoder_block(ebi, filter_i, 3, 2, is_training,
                                scope='eb'+str(i), padding='same')
                
                ebs.append(ebi)
                i = i + 1
            net = ebi# (32,32,512)

            # Decoder
            dbi = decoder_block(net, filters_m[0], filters_n[0], 3,
                                2, is_training=is_training, scope='db4',
                                padding='same')
            i = len(filters_m) - 1
            for filters_i in zip(filters_m[1:-1], filters_n[1:-1]):
                dbi = dbi + ebs[i-1]
                dbi = decoder_block(dbi, filters_i[0], filters_i[1], 3,
                                    2, is_training=is_training,
                                    scope='db'+str(i), padding='same')
                i = i - 1
            dbi = dbi + ebs[0]
            dbi = decoder_block(dbi, filters_m[-1], filters_n[-1], 3, 1,
                                is_training=is_training,
                                scope='db1', padding='same')
            net = dbi

            # Classification
            with tf1.compat.v1.variable_scope('classifier', reuse=reuse):
                net = upconv_bn_relu(net, 32, 3, 2, is_training=is_training,
                                     scope='conv_transpose')
                net = conv_bn_relu(net, 32, 3, 1, is_training=is_training,
                                   scope='conv')
                # Last layer, no batch normalization or activation
                logits = keras.layers.Conv2DTranspose(num_classes,
                                                    kernel_size=2, strides=2,
                                                    padding='same',
                                                    name='conv_transpose',
                                                    kernel_initializer=he_normal())(net)
#                 print("logits.shape",logits.shape) # logits.shape (None, 1024, 1024, 3)
                if num_classes > 1:
                    prob = tf1.nn.softmax(logits, name='prob')
                else:
                    prob = tf1.nn.sigmoid(logits, name='prob')
                    
               

                model = tf1.keras.models.Model(inputs=inputs,outputs=prob)
                return model
                
#                 return prob, logits         

In [ ]:
load_pretrained_weights

# Loss Function and Generator Dataset

dice loss, IDRiD-S+DiaretDB1

# Training and Evaluation

In [ ]:
model = linknet()
vgg_CFL = MODEL()
vgg_CFL.Run_training()
vgg_CFL.Evaluation(num_sample = -1)
del vgg_CFL